## Preprocess data to get training and testing data

I,II,III,aVL,aVR,aVF,V1–V6

In [1]:
# Import package
import time
import numpy as np
import wfdb
import ast
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pylab import mpl
from scipy.fftpack import fft, ifft 
from scipy import signal
# from biosppy.signals import ecg
import neurokit2 as nk
from sklearn import *
from collections import OrderedDict

In [2]:
#Set the read file path
path = '/global/D1/homes/jayao/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.2/ptbxl/'

X = np.load(path + 'raw100_segmented.npy', allow_pickle=True)
sampling_rate = 100

In [3]:
# Read the file and convert tags
Y = pd.read_csv(path+'ptbxl_database.csv', index_col='ecg_id')
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

In [4]:
X.shape

(21801, 100, 12)

In [5]:
# Get diagnostic information in scp_statements.csv
agg_df = pd.read_csv(path+'scp_statements.csv', index_col=0)

In [6]:
agg_df = agg_df[agg_df.diagnostic == 1]

In [7]:
def diagnostic_class(scp):
    res = set()
    for k in scp.keys():
        if k in agg_df.index:
            res.add(agg_df.loc[k].diagnostic_class)
    return list(res)

In [8]:
def aggregate_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_class)
    return list(set(tmp))

In [9]:
Y['scp_classes'] = Y.scp_codes.apply(diagnostic_class)

In [10]:
Z = pd.DataFrame(0, index=Y.index, columns=['NORM', 'MI', 'STTC', 'CD', 'HYP'], dtype='int')
for i in Z.index:
    for k in Y.loc[i].scp_classes:
        Z.loc[i, k] = 1

Z

,NORM,MI,STTC,CD,HYP
ecg_id,,,,,
1,1,0,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0
5,1,0,0,0,0
...,...,...,...,...,...
21833,0,0,1,0,0
21834,1,0,0,0,0
21835,0,0,1,0,0


In [11]:
hyp_cases = Z[Z['HYP'] == 1]
hyp_cases.head(20)

,NORM,MI,STTC,CD,HYP
ecg_id,,,,,
30,0,0,0,0,1
45,0,0,0,1,1
96,0,0,0,0,1
106,0,1,0,0,1
138,0,0,0,0,1
146,0,1,1,0,1
162,0,1,1,1,1
173,0,0,1,0,1
191,0,0,0,1,1


In [12]:
#Add diagnostic information
Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)

In [13]:
Y.diagnostic_superclass.value_counts()

diagnostic_superclass
[NORM]                 9072
[MI]                   2532
[STTC]                 2401
[CD]                   1708
[CD, MI]               1300
[HYP, STTC]             781
[STTC, MI]              600
[HYP]                   535
[STTC, CD]              471
[CD, NORM]              407
[]                      405
[HYP, STTC, MI]         361
[HYP, CD]               300
[STTC, CD, MI]          223
[HYP, STTC, CD]         211
[HYP, MI]               183
[HYP, STTC, CD, MI]     156
[HYP, CD, MI]           117
[STTC, NORM]             28
[STTC, CD, NORM]          5
[HYP, CD, NORM]           2
[HYP, NORM]               2
[HYP, CD, NORM, MI]       1
Name: count, dtype: int64

In [14]:

X_all = X[(Y.strat_fold <= 11)]
X_all.shape

(21801, 100, 12)

In [15]:
save_path = '/global/D1/homes/jayao/XAI-Based-ECG-Diagnostics-main/data/'

In [16]:
# Split data into train and test
test_fold = 10
# # Train
X_train = X[(Y.strat_fold <= 8)]
# y_train = Z[Y.strat_fold <= 8]
y_train = Y[(Y.strat_fold <= 8)].diagnostic_superclass
# # Test
X_test = X[(Y.strat_fold >8)]
# y_test = Z[Y.strat_fold > 8]
y_test = Y[(Y.strat_fold > 8)].diagnostic_superclass


print(X_train.shape, y_train.shape)
print(X_test.shape,  y_test.shape)

(17420, 100, 12) (17420,)
(4381, 100, 12) (4381,)


In [17]:
# Display y_train for rows 100 to 150
print(y_train.iloc[2000:3000])


ecg_id
2691             [STTC]
2695             [NORM]
2696    [HYP, STTC, MI]
2699             [NORM]
2700             [NORM]
             ...       
4035             [NORM]
4036             [NORM]
4037             [NORM]
4038           [CD, MI]
4039             [NORM]
Name: diagnostic_superclass, Length: 1000, dtype: object


In [18]:
save_path = '/global/D1/homes/jayao/XAI-Based-ECG-Diagnostics-main/data/segmented/'

np.save(save_path+'x_train.npy', X_train)
np.save(save_path+'y_train.npy', np.array(y_train))
np.save(save_path+'x_test.npy', X_test)
np.save(save_path+'y_test.npy', np.array(y_test))

## ECG SHaP starts

In [19]:
from tensorflow.keras import layers, optimizers, losses, metrics, activations, regularizers, callbacks
from keras.models import Model
import numpy as np
import pandas as pd
from tensorflow.keras.layers import LSTM

2024-05-05 13:24:53.691586: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-05 13:24:53.691644: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-05 13:24:53.692851: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-05 13:24:54.080404: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-05 13:25:09.681690: W tensorflow/c

In [24]:
path = "/global/D1/homes/jayao/XAI-Based-ECG-Diagnostics-main/data/segmented/"
x_train = np.load(path + 'x_train.npy')
y_train = np.load(path + 'y_train.npy', allow_pickle=True)
x_test  = np.load(path + 'x_test.npy')
y_test  = np.load(path + 'y_test.npy', allow_pickle=True)
print(x_train.shape)

(17420, 100, 12)


In [25]:
x_train = x_train.transpose(0, 2, 1)            # transpose working correctly
x_test  = x_test.transpose(0, 2, 1)
print(x_train.shape)

(17420, 12, 100)


In [26]:
x_train = x_train.reshape(17420, 12, 100, 1)   # Add another channel
x_test  = x_test.reshape(4381, 12, 100, 1)

In [27]:
print("x_train :", x_train.shape)
print("y_train :", y_train.shape)
print("x_test  :", x_test.shape)
print("y_test  :", y_test.shape)
print('Data loaded')

# Old OUTPUTS:
# (19601, 1000, 12)
# (19601, 12, 1000)
# x_train : (19601, 12, 1000, 1)
# y_train : (19601,)
# x_test  : (2198, 12, 1000, 1)
# y_test  : (2198,)
# Data loaded

x_train : (17420, 12, 100, 1)
y_train : (17420,)
x_test  : (4381, 12, 100, 1)
y_test  : (4381,)
Data loaded


In [6]:
x_test.shape

(4381, 12, 1000, 1)

In [28]:

from sklearn.preprocessing import MultiLabelBinarizer
# Convert multi-label target labels to one-hot encoded matrix
mlb = MultiLabelBinarizer()
y_train = mlb.fit_transform(y_train)
y_test = mlb.transform(y_test)
print("Classes:", mlb.classes_)

Classes: ['CD' 'HYP' 'MI' 'NORM' 'STTC']


In [29]:
value_at_index = y_train[6666]
print(value_at_index)

[0 0 0 1 0]


In [30]:
y_train.shape

(17420, 5)

In [31]:
y_test.shape

(4381, 5)

In [38]:
# ST-CNN
# Main Version
input = layers.Input(shape=(12, 100, 1))

X = layers.Conv2D(filters=32, kernel_size=(1, 5))(input)
X = layers.BatchNormalization()(X)
X = layers.ReLU()(X)
X = layers.MaxPooling2D(pool_size=(1, 2), strides=1)(X)

convC1 = layers.Conv2D(filters=64, kernel_size=(1, 7))(X)

X = layers.Conv2D(filters=32, kernel_size=(1, 5))(X)
X = layers.BatchNormalization()(X)
X = layers.ReLU()(X)
X = layers.MaxPooling2D(pool_size=(1, 4), strides=1)(X)

convC2 = layers.Conv2D(filters=64, kernel_size=(1, 6))(convC1)

X = layers.Conv2D(filters=64, kernel_size=(1, 5))(X)
X = layers.BatchNormalization()(X)
X = layers.Add()([convC2, X])           # skip Connection
X = layers.ReLU()(X)
X = layers.MaxPooling2D(pool_size=(1, 2), strides=1)(X)

convE1 = layers.Conv2D(filters=32, kernel_size=(1, 4))(X)

X = layers.Conv2D(filters=64, kernel_size=(1, 3))(X)
X = layers.BatchNormalization()(X)
X = layers.ReLU()(X)
X = layers.MaxPooling2D(pool_size=(1, 4), strides=1)(X)

convE2 = layers.Conv2D(filters=64, kernel_size=(1, 5))(convE1)

X = layers.Conv2D(filters=64, kernel_size=(1, 3))(X)
X = layers.BatchNormalization()(X)
X = layers.Add()([convE2, X])         # skip Connection
X = layers.ReLU()(X)
X = layers.MaxPooling2D(pool_size=(1, 2), strides=1)(X)
print('Added 5 layers for temporal analysis')

X = layers.Conv2D(filters=64, kernel_size=(12, 1))(X)
X = layers.BatchNormalization()(X)
X = layers.ReLU()(X)
X = layers.GlobalAveragePooling2D()(X)
print('Added 1 layer for spatial Analysis')

X = layers.Flatten()(X)
print(X.shape)

X = layers.Dense(units=128, kernel_regularizer=regularizers.L2(0.005))(X)
X = layers.BatchNormalization()(X)
X = layers.ReLU()(X)
X = layers.Dropout(rate=0.1)(X)

X = layers.Dense(units=64, kernel_regularizer=regularizers.L2(0.009))(X)
X = layers.BatchNormalization()(X)
X = layers.ReLU()(X)
X = layers.Dropout(rate=0.15)(X)
print('Added 2 fully connected layers')

output = layers.Dense(5, activation='sigmoid')(X)
model = Model(inputs=input, outputs=output)
print(model.summary())

Added 5 layers for temporal analysis
Added 1 layer for spatial Analysis
(None, 64)
Added 2 fully connected layers
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 12, 100, 1)]         0         []                            
                                                                                                  
 conv2d_10 (Conv2D)          (None, 12, 96, 32)           192       ['input_2[0][0]']             
                                                                                                  
 batch_normalization_8 (Bat  (None, 12, 96, 32)           128       ['conv2d_10[0][0]']           
 chNormalization)                                                                                 
                                                                             

Hyper parameter tuning:
EarlyStopping monitors a specified metric, here: "val_loss"
If the val_loss does not improve for a certain number of epochs defined by patience (in this case, 6 epochs), training is stopped early.
The restore_best_weights=True argument ensures that the weights of the model are restored to the best weights when training stopped.

Learning Rate Reduction:
ReduceLROnPlateau monitors the validation loss ("val_loss").
If the validation loss does not improve for a certain number of epochs defined by patience (in this case, 3 epochs), the learning rate is reduced by a factor defined by factor.

In [32]:
# #Enhanced ST-CNN
# from tensorflow.keras import layers, regularizers
# from tensorflow.keras.models import Model

# # Main Version
# input = layers.Input(shape=(12, 100, 1))

# X = layers.Conv2D(filters=32, kernel_size=(1, 5), padding='same')(input)
# X = layers.BatchNormalization()(X)
# X = layers.ReLU()(X)
# X = layers.MaxPooling2D(pool_size=(1, 2), strides=1, padding='same')(X)

# convC1 = layers.Conv2D(filters=64, kernel_size=(1, 7), padding='same')(X)

# X = layers.Conv2D(filters=32, kernel_size=(1, 5), padding='same')(X)
# X = layers.BatchNormalization()(X)
# X = layers.ReLU()(X)
# X = layers.MaxPooling2D(pool_size=(1, 4), strides=1, padding='same')(X)

# convC2 = layers.Conv2D(filters=64, kernel_size=(1, 6), padding='same')(convC1)

# X = layers.Conv2D(filters=64, kernel_size=(1, 5), padding='same')(X)
# X = layers.BatchNormalization()(X)
# X = layers.ReLU()(X)
# residual_1 = layers.Add()([convC2, X])           # skip Connection
# X = layers.ReLU()(residual_1)
# X = layers.MaxPooling2D(pool_size=(1, 2), strides=1, padding='same')(X)

# convE1 = layers.Conv2D(filters=32, kernel_size=(1, 4), padding='same')(X)

# X = layers.Conv2D(filters=64, kernel_size=(1, 3), padding='same')(X)
# X = layers.BatchNormalization()(X)
# X = layers.ReLU()(X)
# X = layers.MaxPooling2D(pool_size=(1, 4), strides=1, padding='same')(X)

# convE2 = layers.Conv2D(filters=64, kernel_size=(1, 5), padding='same')(convE1)

# X = layers.Conv2D(filters=64, kernel_size=(1, 3), padding='same')(X)
# X = layers.BatchNormalization()(X)
# X = layers.ReLU()(X)
# residual_2 = layers.Add()([convE2, X])         # skip Connection
# X = layers.ReLU()(residual_2)
# X = layers.MaxPooling2D(pool_size=(1, 2), strides=1, padding='same')(X)
# print('Added 5 layers for temporal analysis')

# # Spatial Analysis
# X = layers.Conv2D(filters=64, kernel_size=(12, 1), padding='same')(X)
# X = layers.BatchNormalization()(X)
# X = layers.ReLU()(X)
# X = layers.GlobalAveragePooling2D()(X)
# print('Added 1 layer for spatial Analysis')

# # Fully Connected Layers
# X = layers.Flatten()(X)
# X = layers.Dense(units=128, kernel_regularizer=regularizers.L2(0.005))(X)
# X = layers.BatchNormalization()(X)
# X = layers.ReLU()(X)
# X = layers.Dropout(rate=0.3)(X)

# X = layers.Dense(units=64, kernel_regularizer=regularizers.L2(0.009))(X)
# X = layers.BatchNormalization()(X)
# X = layers.ReLU()(X)
# X = layers.Dropout(rate=0.3)(X)
# print('Added 2 fully connected layers')

# # Output Layer
# output = layers.Dense(5, activation='sigmoid')(X)

# # Define the model
# model = Model(inputs=input, outputs=output)
# print(model.summary())


2024-05-05 13:26:17.716510: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1562 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:5e:00.0, compute capability: 7.5


Added 5 layers for temporal analysis
Added 1 layer for spatial Analysis
Added 2 fully connected layers
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 12, 100, 1)]         0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 12, 100, 32)          192       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 12, 100, 32)          128       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                          

In [39]:
# Source: https://keras.io/api/callbacks/
# Source: https://towardsdatascience.com/checkpointing-deep-learning-models-in-keras-a652570b8de6

early    = callbacks.EarlyStopping(monitor="val_loss", patience=6, restore_best_weights=True)
reducelr = callbacks.ReduceLROnPlateau(monitor="val_loss", patience=3)
callback = [early, reducelr]

# ST-CNN
model.compile(optimizer = optimizers.Adam(learning_rate=0.005),
# #Enhanced ST-CNN
# model.compile(optimizer = optimizers.Adam(learning_rate=0.001),
              loss = losses.BinaryCrossentropy(),
              metrics = [metrics.BinaryAccuracy(), metrics.AUC(curve='ROC', multi_label=True)])

history = model.fit(x_train, y_train, validation_split=0.12, epochs=20, batch_size=64, callbacks=callback)

Epoch 1/20
240/240 [==============================] - 24s 44ms/step - loss: 0.5583 - binary_accuracy: 0.8135 - auc_1: 0.8149 - val_loss: 0.5961 - val_binary_accuracy: 0.7455 - val_auc_1: 0.6065 - lr: 0.0050
Epoch 2/20
240/240 [==============================] - 8s 33ms/step - loss: 0.3973 - binary_accuracy: 0.8470 - auc_1: 0.8616 - val_loss: 0.4752 - val_binary_accuracy: 0.8097 - val_auc_1: 0.8545 - lr: 0.0050
Epoch 3/20
240/240 [==============================] - 8s 34ms/step - loss: 0.3744 - binary_accuracy: 0.8569 - auc_1: 0.8782 - val_loss: 0.5558 - val_binary_accuracy: 0.7374 - val_auc_1: 0.8411 - lr: 0.0050
Epoch 4/20
240/240 [==============================] - 8s 34ms/step - loss: 0.3573 - binary_accuracy: 0.8648 - auc_1: 0.8885 - val_loss: 0.4678 - val_binary_accuracy: 0.8043 - val_auc_1: 0.8569 - lr: 0.0050
Epoch 5/20
240/240 [==============================] - 8s 34ms/step - loss: 0.3479 - binary_accuracy: 0.8677 - auc_1: 0.8944 - val_loss: 0.5665 - val_binary_accuracy: 0.7649 - 

In [40]:
save_path = "/global/D1/homes/jayao/XAI-Based-ECG-Diagnostics-main/model/"
# model.save(save_path + "ST-CNN-5_final1.h5")
model.save(save_path + "ST-CNN-5_final1_segmented.h5")

/global/D1/homes/jayao/.venv/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [41]:
from tensorflow.keras.models import load_model
model = load_model(r'/global/D1/homes/jayao/XAI-Based-ECG-Diagnostics-main/model/ST-CNN-5_final1_segmented.h5')

In [42]:
#ST-CNN
y_pred_train = model.predict(x_train)
y_pred_test  = model.predict(x_test)

from sklearn.metrics import classification_report, precision_recall_curve, f1_score, roc_auc_score, accuracy_score, auc
import numpy as np

def sklearn_metrics(y_true, y_pred, mlb):
    y_bin = np.copy(y_pred)
    y_bin[y_bin >= 0.5] = 1
    y_bin[y_bin < 0.5]  = 0

#     print("y_train shape:", y_true.shape)
# p   print("y_test shape :", y_pred.shape)


    # Compute area under precision-Recall curve
    auc_sum = 0
    for i in range(y_true.shape[1]):
        precision, recall, thresholds = precision_recall_curve(y_true[:, i], y_pred[:, i])
        auc_sum += auc(recall, precision)

    print("Accuracy        : {:.2f}".format(accuracy_score(y_true.flatten(), y_bin.flatten()) * 100))
    print("Macro AUC score : {:.2f}".format(roc_auc_score(y_true, y_pred, average='macro') * 100))
    print('AUROC           : {:.2f}'.format((auc_sum / y_true.shape[1]) * 100))
    print("Micro F1 score  : {:.2f}".format(f1_score(y_true, y_bin, average='micro') * 100))

    # Convert binary predictions back to class labels using MultiLabelBinarizer
    predicted_classes = mlb.inverse_transform(y_bin)

    # Use a set to accumulate all distinct classes
    distinct_classes = set()

    # Iterate over predicted classes and add them to the set
    for classes in predicted_classes:
        distinct_classes.update(classes)

    # Convert the set of distinct classes to a sorted list
    class_names = sorted(list(distinct_classes))

    # Print classification report for each class
    print("\nClassification Report:")
    print(classification_report(y_true, y_bin, target_names=class_names))

# Assuming mlb is the MultiLabelBinarizer used for transforming the labels
sklearn_metrics(y_test, y_pred_test, mlb)


137/137 [==============================] - 1s 8ms/step
Accuracy        : 88.08
Macro AUC score : 91.87
AUROC           : 79.57
Micro F1 score  : 75.48

Classification Report:
              precision    recall  f1-score   support

          CD       0.79      0.68      0.73       992
         HYP       0.75      0.38      0.51       530
          MI       0.81      0.59      0.68      1092
        NORM       0.80      0.92      0.86      1919
        STTC       0.79      0.69      0.74      1049

   micro avg       0.79      0.72      0.75      5582
   macro avg       0.79      0.65      0.70      5582
weighted avg       0.79      0.72      0.74      5582
 samples avg       0.77      0.74      0.74      5582



/global/D1/homes/jayao/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/global/D1/homes/jayao/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [37]:
# #Enhanced ST-CNN
# y_pred_train = model.predict(x_train)
# y_pred_test  = model.predict(x_test)

# from sklearn.metrics import classification_report, precision_recall_curve, f1_score, roc_auc_score, accuracy_score, auc
# import numpy as np

# def sklearn_metrics(y_true, y_pred, mlb):
#     y_bin = np.copy(y_pred)
#     y_bin[y_bin >= 0.5] = 1
#     y_bin[y_bin < 0.5]  = 0

# #     print("y_train shape:", y_true.shape)
# # p   print("y_test shape :", y_pred.shape)


#     # Compute area under precision-Recall curve
#     auc_sum = 0
#     for i in range(y_true.shape[1]):
#         precision, recall, thresholds = precision_recall_curve(y_true[:, i], y_pred[:, i])
#         auc_sum += auc(recall, precision)

#     print("Accuracy        : {:.2f}".format(accuracy_score(y_true.flatten(), y_bin.flatten()) * 100))
#     print("Macro AUC score : {:.2f}".format(roc_auc_score(y_true, y_pred, average='macro') * 100))
#     print('AUROC           : {:.2f}'.format((auc_sum / y_true.shape[1]) * 100))
#     print("Micro F1 score  : {:.2f}".format(f1_score(y_true, y_bin, average='micro') * 100))

#     # Convert binary predictions back to class labels using MultiLabelBinarizer
#     predicted_classes = mlb.inverse_transform(y_bin)

#     # Use a set to accumulate all distinct classes
#     distinct_classes = set()

#     # Iterate over predicted classes and add them to the set
#     for classes in predicted_classes:
#         distinct_classes.update(classes)

#     # Convert the set of distinct classes to a sorted list
#     class_names = sorted(list(distinct_classes))

#     # Print classification report for each class
#     print("\nClassification Report:")
#     print(classification_report(y_true, y_bin, target_names=class_names))

# # Assuming mlb is the MultiLabelBinarizer used for transforming the labels
# sklearn_metrics(y_test, y_pred_test, mlb)


137/137 [==============================] - 1s 8ms/step
Accuracy        : 88.15
Macro AUC score : 92.07
AUROC           : 80.03
Micro F1 score  : 75.73

Classification Report:
              precision    recall  f1-score   support

          CD       0.78      0.70      0.74       992
         HYP       0.67      0.50      0.57       530
          MI       0.83      0.57      0.68      1092
        NORM       0.84      0.86      0.85      1919
        STTC       0.73      0.78      0.75      1049

   micro avg       0.79      0.73      0.76      5582
   macro avg       0.77      0.68      0.72      5582
weighted avg       0.79      0.73      0.75      5582
 samples avg       0.76      0.74      0.74      5582



/global/D1/homes/jayao/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/global/D1/homes/jayao/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
